In [ ]:
%reset -f
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import time

from importlib import reload
#syntax: fugi = reload(fugi)

import functions_giammi as fugi
# fugi = reload(fugi)

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
%config InlineBackend.figure_formats=["png"]

import re

#for the cartesian product
import itertools
from itertools import product

#to smooth the MFPADs
import scipy as sp
from scipy import interpolate
import scipy.ndimage
from scipy.spatial import Delaunay
from scipy.ndimage import gaussian_filter

#to create the mesh
import triangulation as tr
from pyntcloud import PyntCloud, structures

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#to see which classes have been defined so far
# import uproot_methods
# [modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

###550eV - 11.5eV CH11
#update nov Kilian binning (36,18)
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CR_Knov_test.root");en="550eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CL_Knov_test.root")
# fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CR_Knov_test.root")
# fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CL_Knov_test.root")

#546eV - 8.2eV CH11
fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546eV_CR_10800-2350ns_multiCH11.root");en="546eV";CH="CH11";
fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546eV_CL_10800-2350ns_multiCH11.root")

# 544eV - 6.1eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CR_9600-3700ns_multiCH11.root");en="544eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CL_9600-3700ns_multiCH11.root")

# #542eV - 4eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542.5eV_CR_10800-2350ns_COMBI_multiCH11.root");en="542eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542.5eV_CL_10800-2350ns_COMBI_multiCH11.root")

#541eV 3eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541.5eV_CR_10800-2350ns_multiCH11.root");en="541eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541.5eV_CL_10800-2350ns_multiCH11.root")

#TFMOx theoretical data
pathth = r'../PYTHON_graphs/DATA/Theory/TFMox-S_22/enant=S_hel=+1_KE=11.7eV';en="KE=11.7eV";
# pathth = r'../PYTHON_graphs/DATA/Theory/TFMox-S_22/enant=S_hel=+1_KE=8.1eV';en="KE=8.1eV";
# pathth = r'../PYTHON_graphs/DATA/Theory/TFMox-S_22/enant=S_hel=+1_KE=6.1eV';en="KE=6.1eV";
# pathth = r'../PYTHON_graphs/DATA/Theory/TFMox-S_22/enant=S_hel=+1_KE=4.1eV';en="KE=4.1eV";
# pathth = r'../PYTHON_graphs/DATA/Theory/TFMox-S_22/enant=S_hel=+1_KE=3.1eV';en="KE=3.1eV";

path = "angular_distr_el/"+CH+"/"
full_72=False
MFPAD=False

#move first the ID you need
IDs = ["ID_ALL_mol_e0_valid/EN_gate/","ID1_mol_e0_valid/EN_gate/","ID2_mol_e0_valid/EN_gate/","ID3_mol_e0_valid/EN_gate/"]
dirs = ["MFPADs_multinew/","MFPADs_multinew_phi/"]

loc=path+IDs[3]+dirs[0]
ID=str(IDs[3]).split("_")[0]

# glob has to be sorted (both in unix and windows)
all_files = sorted(glob.glob(pathth + "/**.dat"))

li = []; cosphi_th = []; test_opening = []; allnames=[]
colnames = ["phi", "cos(theta)", "value"]

#to alow the helicity comparison without the S enantiomer
if en != "550eV":
    fileSCR = None
    fileSCL = None

#how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/,
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None)  #r"\s+" is a regex (regular expression)
    #or delim_whitespace=True, it is faste
    temp_name=(filename.replace(" ","")).split("/")[-1].split(".")[0] #adding a column to df with the file name with no spaces and extension
    if temp_name.split("_")[0] != 'MFPAD':
        cosphi_th.append((int(temp_name.split("_")[0]), int(temp_name.split("_")[1])))
        allnames.append(temp_name)
        df["filename"]=temp_name
        df["filename"]=df["filename"].astype("category")
        li.append(df) #a unique DataFrame
    else:
        continue

cosphi_th = np.array(cosphi_th)
cosphi_th_adj = np.around(np.array(list(itertools.product(np.linspace(1.,-1.,6).tolist(),np.linspace(-180,180,12).tolist()))),3)
cosphi_th_adj_exp = np.around(np.array(list(itertools.product(np.linspace(0.835,-0.835,6).tolist(),np.linspace(-165,165,12).tolist()))),3)

frame = pd.concat(li, axis=0)
#this commmand avoids 1_1 followed by 1_10: semantic order
frame_srt = frame.groupby("filename",sort=False)
frame_set = frame.set_index("filename") # build a multiindex using the categories of filename

In [ ]:
#Loading of the EXPERIMENTAL MFPADS
MFPAD_RCR,ctheta_RCR,ctheta_RCR_err,cosphi_photon,MFPAD_axis,ctheta_axis,ctheta_axis_red=fugi.import_MFPAD(fileRCR, loc, full=True)
MFPAD_RCL,ctheta_RCL,ctheta_RCL_err=fugi.import_MFPAD(fileRCL, loc)
MFPAD_RCR_phi=fugi.sorting_array(MFPAD_RCR, theory=False, a=2)
MFPAD_RCL_phi=fugi.sorting_array(MFPAD_RCL, theory=False, a=2)
# MFPAD_RCR_norm=fugi.normalise_with_err(MFPAD_RCR_phi,normtype=2)
# MFPAD_RCL_norm=fugi.normalise_with_err(MFPAD_RCL_phi,normtype=2)

if fileSCL and fileSCR is not None:
    MFPAD_SCR,ctheta_SCR,ctheta_SCR_err=fugi.import_MFPAD(fileSCR, loc)
    MFPAD_SCL,ctheta_SCL,ctheta_SCL_err=fugi.import_MFPAD(fileSCL, loc)
    MFPAD_SCR_phi=fugi.sorting_array(MFPAD_SCR, theory=False, a=2)
    MFPAD_SCL_phi=fugi.sorting_array(MFPAD_SCL, theory=False, a=2)
    # MFPAD_SCR_norm=fugi.normalise_with_err(MFPAD_SCR_phi,normtype=2)
    # MFPAD_SCL_norm=fugi.normalise_with_err(MFPAD_SCL_phi,normtype=2)


#COORDIANTES PHOTON ROTATION
#adjusting cosphi with a shifting to have it equal to theory
ctheta_axis_red=ctheta_axis_red[0]; #Ffrom ccos(theta)

phiM=(MFPAD_axis[0][0][1:] + MFPAD_axis[0][0][:-1])/2 #phi = (36,), MFPAD_axis = (37,)
cosM=(MFPAD_axis[0][1][1:] + MFPAD_axis[0][1][:-1])/2 #COS = (18,), MFPAD_axis = (19,)
phiMM, cosMM = np.meshgrid(phiM, cosM)

phicos_phi=list(product(phiM, cosM))
phi_full = np.array([el[0] for el in phicos_phi])
cos_full = np.array([el[1] for el in phicos_phi])

phi_rad= phi_full*np.pi/180.
theta_rad= np.arccos(cos_full)

print("MFPAD_xy shape = ", MFPAD_axis[0][0].shape)
print("len phi_rad = ", phi_rad.shape)
print("len theta_rad = ", theta_rad.shape)
print("phiM shape = ", phiM.shape)
print("cosM shape = ", cosM.shape)

In [ ]:
#EXPERIMENTAL DATA
#smart flipping for the single 72 MFPAD image. Has to be in phi because of the structure of reshape, do the cos just as a check
temp_RCLCR_phi=np.empty([12,6,648])

nphi=MFPAD_RCL.shape[1];ncos=MFPAD_RCL.shape[2]

#sum helicities: -cos(theta), phi+180 in photon coordiantes
for j in range(12):
    for i in range(6):
        if j>5:
            temp_RCLCR_phi[j,i]=np.add(MFPAD_RCL_phi.reshape(12,6,-1)[j,i],MFPAD_RCR_phi.reshape(12,6,-1)[j-6,5-i])
        else:
            temp_RCLCR_phi[j,i]=np.add(MFPAD_RCL_phi.reshape(12,6,-1)[j,i],MFPAD_RCR_phi.reshape(12,6,-1)[6+j,5-i])

#now they are scanned along phi
MFPAD_RCLCR_phi=np.array(temp_RCLCR_phi).reshape(72,nphi,ncos)
MFPAD_RCLCR_phi_norm=fugi.normalise_with_err(MFPAD_RCLCR_phi,normtype=2)

#PA-MFPAD
MFPAD_Rtot_phi=np.sum(MFPAD_RCLCR_phi-MFPAD_RCLCR_phi.min(), axis=0)
MFPAD_Rtot_phi_norm=fugi.normalise_with_err(MFPAD_Rtot_phi,normtype=3)
MFPAD_Rtot_phi_norm2=fugi.normalise_with_err(MFPAD_Rtot_phi,normtype=2)
MFPAD_Rtot_phi_norm2_0=MFPAD_Rtot_phi_norm2-MFPAD_Rtot_phi_norm2.min()
print("R norm 1: min ", MFPAD_Rtot_phi_norm.min(), " max", MFPAD_Rtot_phi_norm.max())
print("R norm 2: min ", MFPAD_Rtot_phi_norm2.min(), " max", MFPAD_Rtot_phi_norm2.max())
print("R norm 2_0: min ", MFPAD_Rtot_phi_norm2_0.min(), " max", MFPAD_Rtot_phi_norm2_0.max())

if fileSCL and fileSCR is not None:
    temp_totordered_phi=np.empty([12,6,648])
    temp_SCLCR_phi=np.empty([12,6,648])
    temp1=[]; tempf=[]
    for el in MFPAD_RCLCR_phi:
        temp1.append(np.flip(el,axis=1))
    MFPAD_RCLCR_phi_flip=np.array(temp1).reshape(72,nphi,ncos)

    for el in temp_RCLCR_phi.reshape(72,nphi,ncos):
        tempf.append(np.flip(el,axis=1))
    temp_RCLCR_phi_flip=np.array(tempf).reshape(12,6,-1)
    
    #sum enantiomer: -cos(theta) fliped axis=1 cos!, -cos(theta) photon coordinates

    for j in range(12):
        for i in range(6):
            if j>5:
                temp_SCLCR_phi[j,i]=np.add(MFPAD_SCL_phi.reshape(12,6,-1)[j,i],MFPAD_SCR_phi.reshape(12,6,-1)[j-6,5-i])

            else:
                temp_SCLCR_phi[j,i]=np.add(MFPAD_SCL_phi.reshape(12,6,-1)[j,i],MFPAD_SCR_phi.reshape(12,6,-1)[6+j,5-i])
    #adding S and R directly to improve statistics
    for j in range(12):
        for i in range(6):
            temp_totordered_phi[j,i]=np.add(temp_SCLCR_phi[j,i],gaussian_filter(temp_RCLCR_phi_flip, sigma=0.2)[j,i])
            # temp_totordered_phi[j,i]=np.add(temp_SCLCR_phi[j,i],gaussian_filter(temp_RCLCR_phi_flip, sigma=0.3)[j,5-i])
    
    MFPAD_SCLCR_phi=np.array(temp_SCLCR_phi).reshape(72,nphi,ncos)
    MFPAD_SCLCR_phi_norm=fugi.normalise_with_err(MFPAD_SCLCR_phi,normtype=2)
    MFPAD_S_sum_phi=np.sum(MFPAD_SCLCR_phi, axis=0)
    
    #all 72
    MFPAD_SR_phi=np.array(temp_totordered_phi).reshape(72,nphi,ncos)
    MFPAD_SR_phi_norm=fugi.normalise_with_err(MFPAD_SR_phi,normtype=2)

    #PA-MFPAD
    MFPAD_Stot_phi=np.sum(MFPAD_SR_phi-MFPAD_SR_phi.min(), axis=0)
    MFPAD_Stot_phi_norm=fugi.normalise_with_err(MFPAD_Stot_phi,normtype=3)
    MFPAD_Stot_phi_norm2=fugi.normalise_with_err(MFPAD_Stot_phi,normtype=2)
    MFPAD_Stot_phi_norm2_0=MFPAD_Stot_phi_norm2-MFPAD_Stot_phi_norm2.min()
    print("S norm 1: min ", MFPAD_Stot_phi_norm.min(), " max", MFPAD_Stot_phi_norm.max())
    print("S norm 2: min ", MFPAD_Stot_phi_norm2.min(), " max", MFPAD_Stot_phi_norm2.max())
    print("S norm 2_0: min ", MFPAD_Stot_phi_norm2_0.min(), " max", MFPAD_Stot_phi_norm2_0.max())



In [ ]:
#scale up experimental to theoretical (density 200,100)
#y -> cos_exp [-1,1] in 100 = 0.02 steps
#x -> phi_exp [-180,180] in 200 = 1,8 steps
#!gaussian before to avoid noise! the values is different between pure R and S+R
phi_exp_up, cos_exp_up, MFPAD_R_exp_up = fugi.plot_interpolation(phiM,cosM,MFPAD_Rtot_phi.T,upscale=True,after=False,gaussian=3.,xstep=(phiM.max()-phiM.min())/200,ystep=(cosM.max()-cosM.min())/100,ax=None,cont=False)
phi_exp_up, cos_exp_up, MFPAD_R_exp_up_norm2 = fugi.plot_interpolation(phiM,cosM,MFPAD_Rtot_phi_norm2.T,upscale=True,after=False,gaussian=3.,xstep=(phiM.max()-phiM.min())/200,ystep=(cosM.max()-cosM.min())/100,ax=None,cont=False)
phi_exp_up, cos_exp_up, MFPAD_R_exp_up_norm2_0 = fugi.plot_interpolation(phiM,cosM,MFPAD_Rtot_phi_norm2_0.T,upscale=True,after=False,gaussian=3.,xstep=(phiM.max()-phiM.min())/200,ystep=(cosM.max()-cosM.min())/100,ax=None,cont=False)
print("MFPAD Rth shape ", MFPAD_R_exp_up_norm2_0.shape)
print("phi ", len(phi_exp_up), " cos ", len(cos_exp_up))
if fileSCL and fileSCR is not None:
    phi_exp_up, cos_exp_up, MFPAD_S_exp_up = fugi.plot_interpolation(phiM,cosM,MFPAD_Stot_phi.T,upscale=True,after=False,gaussian=2.3,xstep=(phiM.max()-phiM.min())/200,ystep=(cosM.max()-cosM.min())/100,ax=None,cont=False)
    phi_exp_up, cos_exp_up, MFPAD_S_exp_up_norm2 = fugi.plot_interpolation(phiM,cosM,MFPAD_Stot_phi_norm2.T,upscale=True,after=False,gaussian=2.3,xstep=(phiM.max()-phiM.min())/200,ystep=(cosM.max()-cosM.min())/100,ax=None,cont=False)
    phi_exp_up, cos_exp_up, MFPAD_S_exp_up_norm2_0 = fugi.plot_interpolation(phiM,cosM,MFPAD_Stot_phi_norm2_0.T,upscale=True,after=False,gaussian=2.3,xstep=(phiM.max()-phiM.min())/200,ystep=(cosM.max()-cosM.min())/100,ax=None,cont=False)
    print("MFPAD Sth shape ", MFPAD_S_exp_up_norm2_0.shape)

In [ ]:

fig,ax= plt.subplots(1, figsize=(12, 10))
if fileSCL and fileSCR is not None:
    # cs,ax=fugi.plot_interpolation(phi_exp_up,cos_exp_up,MFPAD_S_exp_up_norm2_0.T, ax=ax)
    cs=plt.pcolormesh(phi_exp_up,cos_exp_up,MFPAD_S_exp_up_norm2_0,cmap="viridis")
    ax.set_title("Total MFPAD S "+en)
else:
    # cs,ax=fugi.plot_interpolation(phi_exp_up,cos_exp_up,MFPAD_S_exp_up_norm2_0.T, ax=ax)
    cs=plt.pcolormesh(phi_exp_up,cos_exp_up,MFPAD_R_exp_up_norm2_0,cmap="viridis")
    ax.set_title("Total MFPAD R "+en)

contrast=cs.get_array().max()/cs.get_array().min() #todo it is divided by zero!
aspect=(phi_exp_up.max()/cos_exp_up.max())

ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
ax.set_aspect(aspect)
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
if fileSCL and fileSCR is not None:
    cbar.set_ticks([MFPAD_S_exp_up_norm2_0.min(),MFPAD_S_exp_up_norm2_0.max()])
else:
    cbar.set_ticks([MFPAD_R_exp_up_norm2_0.min(),MFPAD_R_exp_up_norm2_0.max()])

cbar.set_ticklabels([1,f"{contrast:.2f}"])
cbar.ax.set_ylabel('contrast')

plt.imshow(np.sum(temp_RCLCR_phi_flip.reshape(72,-1),axis=0).reshape(36,18).T)

fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_SCLCR_phi[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)
fugi.overlaygraph(fig)

fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_RCLCR_phi[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)
fugi.overlaygraph(fig)

In [ ]:
#THEORY
#NOTE here the cos(theta)_el is sorted in the range [-1:-1]
#TODO create the R enantiomer for energy != 550eV
counts_temp_cosord=[]
for group in frame_srt:
    counts_temp_cosord.append(frame_srt.get_group(group[0]).sort_values(by=["phi","cos(theta)"])["value"].values)
    if group[0] == "1_1":
        phi_full_sorted=np.array(frame_srt.get_group(group[0]).sort_values(by=["phi","cos(theta)"])["phi"].values)
        ctheta_full_sorted=np.array(frame_srt.get_group(group[0]).sort_values(by=["phi","cos(theta)"])["cos(theta)"].values)

counts_temp_cosord=np.array(counts_temp_cosord).reshape(72,200,100)

#NOTE these will be sorted according to cos(theta)_el [-1:1]
phiM_th=np.unique(phi_full_sorted)
cosM_th=np.unique(ctheta_full_sorted)

cos_lin=True; limits=[-1+1/6, 1-1/6, -180+33/2, 180-33/2] #linear in ctheta [-1,1], default with Philipp
xgo_phi,ygo_phi,cosphi_adj_phi = fugi.create_gocoords(a=0, cos_lin=cos_lin, limits=limits, source=True)
xgo_cos_flip,ygo_cos_flip,cosphi_adj_cos_flip = fugi.create_gocoords(a=2, cos_lin=cos_lin, limits=limits, source=True)

dfind=pd.DataFrame(cosphi_adj_cos_flip, columns=["ctheta","phi"])
dfind["item"]=allnames
cosphi_th_std=(dfind.sort_values(by=["ctheta", "phi"], ascending=[False,True]))["item"].values

#NOTE these will be sorted according to cos(theta) or phi PHOTON
# MFPAD_Sth_cos=fugi.sorting_array(counts_temp_cosord, theory=True, items=allnames, a=1)

#MFPAD_S_phi: in photon_coordiantes as allnames 
# OUTPUT a=2 : 6_1', '5_1', '4_1', '3_1', '2_1', '1_1', '6_2', '5_2'
#72 MFPAD
MFPAD_Sth_phi=fugi.sorting_array(counts_temp_cosord, theory=True, items=allnames, a=2)

#PA-MFPAD
MFPAD_Stot_th_phi=np.sum(MFPAD_Sth_phi, axis=0)
#NOTE *10 to have the same order of magnitude as experiments, but fixing the minumum to 0, it is a stretch!
MFPAD_Stot_th_phi_norm2=(fugi.normalise_with_err(MFPAD_Stot_th_phi-MFPAD_Stot_th_phi.min(),normtype=2))*10
MFPAD_Stot_th_phi_norm2_0=(MFPAD_Stot_th_phi_norm2-MFPAD_Stot_th_phi_norm2.min())

MFPAD_Rtot_th_phi=np.flip(MFPAD_Stot_th_phi, axis=1)
MFPAD_Rtot_th_phi_norm2=np.flip(MFPAD_Stot_th_phi_norm2, axis=1)
MFPAD_Stot_th_phi_norm2_0=np.flip(MFPAD_Stot_th_phi_norm2_0, axis=1)

print("MFPAD th shape ", MFPAD_Sth_phi.shape)
print("phi ", len(phiM_th), " cos ", len(cosM_th))
print("phi_full_sorted shape ", phi_full_sorted.shape)
print("norm 1: min ", MFPAD_Stot_th_phi.min(), " max", MFPAD_Stot_th_phi.max())
print("norm 2: min ", MFPAD_Stot_th_phi_norm2.min(), " max", MFPAD_Stot_th_phi_norm2.max())
print("norm 2_0: min ", MFPAD_Stot_th_phi_norm2_0.min(), " max", MFPAD_Stot_th_phi_norm2_0.max())

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
# cs=ax.contourf(phiM,cosM,MFPAD_R_cos_sum.reshape(200,100).T,cmap=cmap_temp)
# cs=ax.contourf(phiMM,cosMM,MFPAD_R_cos_sum.reshape.T, cmap=cmap_temp)
# limits=[0.07,0.25]
if fileSCL and fileSCR is not None:
    cs,ax=fugi.plot_interpolation(phiM_th,cosM_th,MFPAD_Stot_th_phi.T, ax=ax)
    ax.set_title("Total MFPAD S "+en)
else:
    cs,ax=fugi.plot_interpolation(phiM_th,cosM_th,MFPAD_Rtot_th_phi.T, ax=ax)
    ax.set_title("Total MFPAD R "+en)

contrast=cs.get_array().max()/cs.get_array().min() #todo it is divided by zero!
aspect=(phiM_th.max()/cosM_th.max())

ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
ax.set_aspect(aspect)
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
#cbar for contour
cbar.set_ticks([MFPAD_Stot_th_phi.min(),MFPAD_Stot_th_phi.max()])
cbar.set_ticklabels([1,f"{contrast:.2f}"])
cbar.ax.set_ylabel('contrast')

In [ ]:
#describe a set of alpha, beta, gamma angles to rotate the theoretical distribution (would be better to rotate before downsampling)
#general
arr=np.arange(-25,25,5)
arrc=np.arange(-15,15,5)
#550eV optimization range
# arr=np.arange(12,17,0.7)
# arrc=np.arange(2,7,0.7)
#546eV optimization range
# arr=np.arange(12,17,0.7)
# arrc=np.arange(-1.4,1.4,0.7)
#test
# arr=np.arange(-1,1,1)
# arrc=np.arange(-1,1,1)

#alpha, beta, gamma for 3D rotation
abg=list(product(arr, arr, arrc))

a = np.array([el[0] for el in abg])
b = np.array([el[1] for el in abg])
g = np.array([el[2] for el in abg])

print("len(abg) = ",len(abg))

for i,el in enumerate(abg):
    if el == (1,1,1) or el == (-1,-1,-1) or np.all(np.isclose(el, [0.2, 0.2, 0.2], rtol=1e-05)) or el == (0,0,0):
        zeropoint=i

print(abg[zeropoint])
print(f'at {zeropoint:d}')

%store MFPAD_Stot_phi_red
%store theta_rad
%store phi_rad
%store phiMM
%store phiM
%store cosMM
%store cosM

                  | Multi-args   Concurrence    Blocking     Ordered-results
----------------------------------------------------------------------------
Pool.map          | no           yes            yes          yes
Pool.map_async    | no           yes            no           yes
Pool.apply        | yes          no             yes          no
Pool.apply_async  | yes          yes            no           no
Pool.starmap      | yes          yes            yes          yes
Pool.starmap_async| yes          yes            no           no

from: https://stackoverflow.com/questions/8533318/multiprocessing-pool-when-to-use-apply-apply-async-or-map

In [ ]:
#MULTICORE
import multiprocessing

def rotateMFPAD(angle):
# def rotateMFPAD(xyzm, abg, i, Xn, Yn, MFPAD_S_exp_up_norm2_0):
    d_temp=[];
    l2_temp=[];
    x_LF, y_LF, z_LF = np.einsum('ik, kj -> ij', fugi.rot3d(angle[0]*np.pi/180.,angle[1]*np.pi/180.,angle[2]*np.pi/180.,convention=1).T, xyzm)
    mag_LF = np.sqrt(x_LF**2+y_LF**2+z_LF**2)
    ctheta_rot=z_LF/mag_LF #!check if it has any NaN inside
    phi_rot=np.arctan2(y_LF,x_LF)*180./np.pi
    f = interpolate.bisplrep(ctheta_rot, phi_rot, mag_LF,w=mag_LF**-0.5, s=np.std(mag_LF)) #! this should be more dynamic control over w and s
    # f = interpolate.bisplrep(ctheta_rot, phi_rot, mag_LF,s=0.01) #! check value of s
    r_rot=(interpolate.bisplev(Yn[:,0],Xn[0,:],f).T)
    r_rot_norm=fugi.normalise_with_err(np.array(r_rot-r_rot.min()),normtype=2)
    #difference between the rotated theoretical MFPAD and the experimental with enhanced density (up)
    temp1=np.sqrt(((r_rot_norm*10).reshape(-1)-MFPAD_r_tot.reshape(-1))**2)
    l2_temp.append(temp1)
    # d_temp.append(np.sqrt(np.sum(((r_rot_norm*10).reshape(-1)-MFPAD_S_exp_up_norm2_0.reshape(-1))**2))) #old
    d_temp.append(np.sum(temp1))
    # print(f"{i:0d} cycle in {time.perf_counter() - tic:0.4f} seconds")
    return (d_temp, l2_temp)
    
#TODO find a new way to take account of the varibility of each MFPAD on the interpolation
#refit theory
# cos_newfull, phi_newfull = np.mgrid[-1:1:100j, -180:180:200j]
phi_new = np.linspace(phi_full_sorted.min(), phi_full_sorted.max(), 200)
cos_new = np.linspace(ctheta_full_sorted.min(), ctheta_full_sorted.max(), 100)
Xn, Yn = np.meshgrid(phi_new, cos_new)

#theory full scale into cartesian
phi_full_sorted_rad = phi_full_sorted/180.*np.pi
theta_full_sorted_rad = np.arccos(ctheta_full_sorted)

if fileSCL and fileSCR is not None:
    MFPAD_original=MFPAD_Stot_th_phi
    MFPAD_r_tot=MFPAD_S_exp_up_norm2_0
else:
    MFPAD_original=MFPAD_Rtot_th_phi
    MFPAD_r_tot=MFPAD_R_exp_up_norm2_0

#!normalization while comparing, not here! Having min = 0 gives some z_LF = 0
x = MFPAD_original.reshape(-1) * np.sin(theta_full_sorted_rad) * np.cos(phi_full_sorted_rad)
y = MFPAD_original.reshape(-1) * np.sin(theta_full_sorted_rad) * np.sin(phi_full_sorted_rad)
z = MFPAD_original.reshape(-1) * np.cos(theta_full_sorted_rad)
xyzm = np.stack((x, y, z))

pool = multiprocessing.Pool(8) #number of cores in hyperthread
tic = time.perf_counter()
#1st method
# https://stackoverflow.com/questions/39893635/returning-multiple-lists-from-pool-map-processes
# https://stackoverflow.com/questions/5442910/how-to-use-multiprocessing-pool-map-with-multiple-arguments
d_temp, l2_temp = zip(*pool.map(rotateMFPAD, abg))
pool.close()
pool.join()
toc = time.perf_counter()
print(f"All cycles in {toc - tic:0.4f} seconds")

d_single_norm=fugi.normalise_with_err(np.array(d_temp),normtype=2)

print("d_sum shape = ", d_single_norm.shape)

d_single_complete_norm=np.column_stack((a,b,g,d_single_norm)).reshape(-1,4)

d_min_norm=d_single_norm.min()
delta=d_single_norm.max()-d_single_norm.min()

output=[]
for i,el in enumerate(d_single_complete_norm):
    if el[3] <= d_min_norm+delta*0.03:
        output.append(np.append(el,i))
sort_out=sorted(output, key = lambda x:x[3])

print("For the PA-MFPAD")
print(sort_out)

In [ ]:
#SINGLE CORE
d_temp=[];
l2_temp=[];
#TODO find a new way to take account of the varibility of each MFPAD on the interpolation

if fileSCL and fileSCR is not None:
    MFPAD_original=MFPAD_Stot_th_phi
    MFPAD_r_tot=MFPAD_S_exp_up_norm2_0
else:
    MFPAD_original=MFPAD_Rtot_th_phi
    MFPAD_r_tot=MFPAD_R_exp_up_norm2_0

#refit theory
# cos_newfull, phi_newfull = np.mgrid[-1:1:100j, -180:180:200j]
phi_new = np.linspace(phi_full_sorted.min(), phi_full_sorted.max(), 200)
cos_new = np.linspace(ctheta_full_sorted.min(), ctheta_full_sorted.max(), 100)
Xn, Yn = np.meshgrid(phi_new, cos_new)

#theory full scale into cartesian
phi_full_sorted_rad = phi_full_sorted/180.*np.pi
theta_full_sorted_rad = np.arccos(ctheta_full_sorted)

#!normalization while comparing, not here! Having min = 0 gives some z_LF = 0
x = MFPAD_original.reshape(-1) * np.sin(theta_full_sorted_rad) * np.cos(phi_full_sorted_rad)
y = MFPAD_original.reshape(-1) * np.sin(theta_full_sorted_rad) * np.sin(phi_full_sorted_rad)
z = MFPAD_original.reshape(-1) * np.cos(theta_full_sorted_rad)
xyzm = np.stack((x, y, z))

print("Start!")
tic = time.perf_counter()

for i,angle in enumerate(abg):
    x_LF, y_LF, z_LF = np.einsum('ik, kj -> ij', fugi.rot3d(angle[0]*np.pi/180.,angle[1]*np.pi/180.,angle[2]*np.pi/180.,convention=1).T, xyzm)
    if i==0:
        toc_rotcycle = time.perf_counter()
        print(f"Rotation in {toc_rotcycle - tic:0.4f} seconds")
    mag_LF = np.sqrt(x_LF**2+y_LF**2+z_LF**2)
    ctheta_rot=z_LF/mag_LF #!check if it has any NaN inside
    phi_rot=np.arctan2(y_LF,x_LF)*180./np.pi
    f = interpolate.bisplrep(ctheta_rot, phi_rot, mag_LF,w=mag_LF**-0.5, s=np.std(mag_LF)) #! this should be more dynamic control over w and s
    # f = interpolate.bisplrep(ctheta_rot, phi_rot, mag_LF,s=0.01) #! check value of s
    r_rot=(interpolate.bisplev(Yn[:,0],Xn[0,:],f).T)
    if i==0:
        toc_interpcycle = time.perf_counter()
        print(f"Interpolation in {toc_interpcycle - tic:0.4f} seconds")
    r_rot_norm=fugi.normalise_with_err(np.array(r_rot-r_rot.min()),normtype=2)
    #difference between the rotated theoretical MFPAD and the experimental with enhanced density (up)
    l2_temp.append(np.sqrt(((r_rot_norm*10).reshape(-1)-MFPAD_r_tot.reshape(-1))**2))
    # d_temp.append(np.sqrt(np.sum(((r_rot_norm*10).reshape(-1)-MFPAD_S_exp_up_norm2_0.reshape(-1))**2))) #old
    d_temp.append(np.sum(l2_temp[i]))
    # print(f"{i:0d} cycle in {time.perf_counter() - tic:0.4f} seconds")
    if i == 0:
        toc_1cycle = time.perf_counter()
        print(f"One cycle in {toc_1cycle - tic:0.4f} seconds")
    print("Number of cycle = ", i)

toc_allcycle = time.perf_counter()
print(f"All cycles in {toc_allcycle - tic:0.4f} seconds")

d_single_norm=fugi.normalise_with_err(np.array(d_temp),normtype=2)

print("d_sum shape = ", d_single_norm.shape)

d_single_complete_norm=np.column_stack((a,b,g,d_single_norm)).reshape(-1,4)

d_min_norm=d_single_norm.min()
delta=d_single_norm.max()-d_single_norm.min()

output=[]
for i,el in enumerate(d_single_complete_norm):
    if el[3] <= d_min_norm+delta*0.03:
        output.append(np.append(el,i))
sort_out=sorted(output, key = lambda x:x[3])

print("For the PA-MFPAD")
print(sort_out)

In [ ]:
# plt.pcolormesh(l2_temp[2].reshape(100,200), cmap="viridis")
plt.pcolormesh(r_rot_norm.reshape(200,100).T, cmap="viridis")

In [ ]:
#rotated MFPAD
fig,ax= plt.subplots(1, figsize=(12, 10))
# cs=ax.contourf(phiM,cosM,MFPAD_R_cos_sum.reshape(200,100).T,cmap=cmap_temp)
# cs=ax.contourf(phiMM,cosMM,MFPAD_R_cos_sum.reshape.T, cmap=cmap_temp)
if fileSCL and fileSCR is not None:
    MFPAD_Stot_th_phi_rot=fugi.rot3d_MFPAD_dist(MFPAD_Stot_th_phi,theta_full_sorted_rad,phi_full_sorted_rad,np.array([tuple(sort_out[0][0:3])]),Xn[0,:],Yn[:,0], convention=1)
    cs=plt.pcolormesh(phiM_th,cosM_th,MFPAD_Stot_th_phi_rot.T,cmap="viridis")
    # cs,ax=fugi.plot_interpolation(phiM_th,cosM_th,MFPAD_Stot_th_phi_rot.T, ax=ax)
    ax.set_title("Total MFPAD S "+en)
    cbar.set_ticks([MFPAD_Stot_th_phi_rot.min(),MFPAD_Stot_th_phi_rot.max()])
else:
    MFPAD_Rtot_th_phi_rot=fugi.rot3d_MFPAD_dist(MFPAD_Rtot_th_phi,theta_full_sorted_rad,phi_full_sorted_rad,np.array([tuple(sort_out[0][0:3])]),Xn[0,:],Yn[:,0], convention=1)
    cs=plt.pcolormesh(phiM_th,cosM_th,MFPAD_Rtot_th_phi_rot.T,cmap="viridis")
    # cs,ax=fugi.plot_interpolation(phiM_th,cosM_th,MFPAD_Stot_th_phi_rot.T, ax=ax)
    ax.set_title("Total MFPAD R "+en)
    cbar.set_ticks([MFPAD_Rtot_th_phi_rot.min(),MFPAD_Rtot_th_phi_rot.max()])

contrast=cs.get_array().max()/cs.get_array().min() #todo it is divided by zero!

aspect=(phiM_th.max()/cosM_th.max())

ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
ax.set_aspect(aspect)
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
#cbar for contour
cbar.set_ticklabels([1,f"{contrast:.2f}"])
cbar.ax.set_ylabel('contrast')

new=[]
for el in MFPAD_rotated_norm:
    for el1 in el:
        new.append(np.nan_to_num(el1.reshape(-1),nan=np.average(np.nan_to_num(el1.reshape(-1)))))

MFPAD_rotated_norm=new

#it works but it takes HOURS!!
import matplotlib.animation as ani
fig = plt.figure()

def animate(i=int):
    p=plt.imshow(MFPAD_Stot_phi_rotated_norm[0][i].T, cmap="viridis")
    return p

animator = ani.FuncAnimation(fig, animate, frames = len(abg))
animator.save("OUTPUTS/test.mp4", fps =30, writer="ffmpeg")
plt.close()

In [ ]:
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, projection='3d')
# img= ax.scatter(a, b, g, c=d_single_norm, alpha=0.8, s=60, cmap=cmap_temp)

d_masked = np.ma.masked_array(d_single_norm, d_single_norm>=d_single_norm.min()+delta*0.8)
img = ax.scatter(a, b, g, c=d_masked, alpha=0.9, s=60, cmap=cmap_temp)

# d_masked = np.ma.masked_array(d_single_norm, d_single_norm>=d_single_norm.min()+delta*0.5)
# img = ax.scatter(a, b, g, c=d_masked, alpha=0.5, s=60, cmap=cmap_temp)

# d_masked = np.ma.masked_array(d_single_norm, d_single_norm>=d_single_norm+delta*0.7)
# img= ax.scatter(a, b, g, c=d_masked, alpha=0.8, s=60, cmap=cmap_temp)

ax.set_xlabel('x alpha [DEG]', labelpad=25)
ax.set_ylabel('y beta [DEG]', labelpad=25)
ax.set_zlabel('z gamma [DEG]', labelpad=25)

ax.view_init(elev=30., azim=45.)

fig.colorbar(img)
plt.show()

In [ ]:
#if d_norm 
fig1 = go.Figure(data=[go.Scatter3d(x=a, y=b, z=g, mode='markers',
                    # marker=dict( size=5,color=d_masked, colorscale='temps', opacity=1., showscale=True)
                    marker=dict( size=5,color=d_single_norm, colorscale='temps', opacity=1., showscale=True)
                    #color set colours to an array/list of desired values
                    #show scale to show the legend according to the color
                    )])

#fig1.update_layout(title="TFMeOx MFPADs EXP", width=500, height=500, margin=dict(l=65, r=50, b=65, t=90)) #margin=dict(l=0, r=0, b=0, t=0))

#fig1.add_trace(go.Surface(x=a, y=b, z=g, colorscale="blues",  showscale=False))
##fig1.add_trace(go.Surface(x=x.apply(lambda x: 10), y=y, z = np.array([z]*length_data), colorscale= bright_blue, showscale=False))
##fig1.add_trace(go.Surface(x=x, y= y.apply(lambda x: 30), z =  np.array([z]*length_data).transpose(), colorscale=bright_pink, showscale=False))
fig1.show()

fig1 = go.Figure(data=[go.Scatter3d(x=d_tot_s[:,0], y=d_tot_s[:,1], z=d_tot_s[:,2], mode='markers',
                    marker=dict( size=5,color=d_tot_s[:,3], colorscale='temps', opacity=.8, showscale=True)
                    #color set colours to an array/list of desired values  
                    #show scale to show the legend according to the color
                    )])

#fig1.update_layout(title="TFMeOx MFPADs EXP", width=500, height=500, margin=dict(l=65, r=50, b=65, t=90)) #margin=dict(l=0, r=0, b=0, t=0))
fig1.show()

from scipy import interpolate

f_th = interpolate.interp2d(cosM_th,phiM_th,MFPAD_Sth_phi[71], kind='cubic')
f_exp = interpolate.interp2d(cosM,phiM,MFPAD_totordered_phi[71], kind='cubic')

ynew = np.linspace(-180, 180, 36)
xnew = np.linspace(-1, 1, 18)
znew = f_th(xnew,ynew)
plt.imshow(znew.T, cmap="viridis")
plt.show()

from scipy.spatial.transform import Rotation, RotationSpline
q = Rotation.from_euler('ZYX', abg, degrees=True)
spline = RotationSpline(znew.reshape(-1), q)